In [1]:
using Pkg
Pkg.activate("../")

ENV["PYTHON"] = "/home/codespace/.cache/pypoetry/virtualenvs/algo-trading-notes-oovsq9PL-py3.11"
Pkg.build("PyCall")


  Activating project at `/workspaces/algo-trading-notes`
    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/51cab8e982c5b598eea9c8ceaced4b58d9dd37c9/build.log`
    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/9816a3826b0ebf49ab4926e2b18842ad8b5c8f04/build.log`


In [2]:
using YFinance
using Dates
using DataFrames
using GLM
using Statistics

In [3]:
# using both range (relative to today) or startdt & enddt works
get_prices("AAPL",startdt="2023-01-30",enddt="2024-01-30",interval="1d")

OrderedCollections.OrderedDict{String, Any} with 8 entries:
  "ticker"    => "AAPL"
  "timestamp" => [DateTime("2023-01-30T14:30:00"), DateTime("2023-01-31T14:30:0…
  "open"      => [144.154, 141.907, 143.17, 148.072, 147.207, 151.722, 149.803,…
  "high"      => [144.741, 143.538, 145.795, 150.34, 156.505, 152.249, 154.367,…
  "low"       => [142.056, 141.489, 140.534, 147.346, 147.008, 149.942, 149.803…
  "close"     => [143.0, 144.29, 145.43, 150.82, 154.5, 151.73, 154.65, 151.92,…
  "adjclose"  => [142.205, 143.488, 144.622, 149.982, 153.641, 150.887, 153.79,…
  "vol"       => [6.36595e7, 6.55083e7, 7.72319e7, 1.17681e8, 1.53499e8, 6.947e…

In [4]:
first(DataFrame(get_prices("AAPL",range="1y",interval="1d")),10)

Row,ticker,timestamp,open,high,low,close,adjclose,vol
,String,DateTime,Float64,Float64,Float64,Float64,Float64,Float64
1,AAPL,2023-01-31T14:30:00,141.907,143.538,141.489,144.29,143.488,6.55083e7
2,AAPL,2023-02-01T14:30:00,143.17,145.795,140.534,145.43,144.622,7.72319e7
3,AAPL,2023-02-02T14:30:00,148.072,150.34,147.346,150.82,149.982,1.17681e8
4,AAPL,2023-02-03T14:30:00,147.207,156.505,147.008,154.5,153.641,1.53499e8
5,AAPL,2023-02-06T14:30:00,151.722,152.249,149.942,151.73,150.887,6.947e7
6,AAPL,2023-02-07T14:30:00,149.803,154.367,149.803,154.65,153.79,8.28595e7
7,AAPL,2023-02-08T14:30:00,153.025,153.721,150.33,151.92,151.076,6.37637e7
8,AAPL,2023-02-09T14:30:00,152.925,153.472,149.584,150.87,150.031,5.56958e7
9,AAPL,2023-02-10T14:30:00,148.856,150.729,148.617,151.01,150.4,5.72186e7


In [5]:
function get_stock_prices(ticker::String, startdt::String, enddt::String, interval::String)
    data_ordered_dict = get_prices(ticker, startdt=startdt, enddt=enddt, interval=interval)
    df = DataFrame(data_ordered_dict)
    return df
end

get_stock_prices (generic function with 1 method)

In [6]:
dataframes_dict = Dict{String, DataFrame}()

Dict{String, DataFrame}()

In [7]:
tickerSymbols = ["META", "AAPL", "AMZN", "NFLX", "GOOGL","^GSPC"];
MAANG = ["META", "AAPL", "AMZN", "NFLX", "GOOGL"];

In [8]:
for tick in tickerSymbols
    temp_df = get_stock_prices(tick, "2023-01-30", "2024-01-30", "1d")
    subset_df = select(temp_df, [:ticker, :timestamp, :close])
    subset_df[!, "Return"] = vcat(NaN, diff(subset_df[!, "close"]) ./ subset_df[1:end-1, "close"])
    dataframes_dict[tick] = subset_df
end

In [9]:
dataframes_dict

Dict{String, DataFrame} with 6 entries:
  "GOOGL" => 251×4 DataFrame…
  "META"  => 251×4 DataFrame…
  "AMZN"  => 251×4 DataFrame…
  "^GSPC" => 251×4 DataFrame…
  "NFLX"  => 251×4 DataFrame…
  "AAPL"  => 251×4 DataFrame…

In [10]:
for ticker in MAANG
    dataframes_dict[ticker][!,"GSPC_Return"] = dataframes_dict["^GSPC"][!,"Return"]
    dataframes_dict[ticker] = filter(row -> isfinite(row[:Return]) && isfinite(row[:GSPC_Return]), dataframes_dict[ticker])
end

In [11]:
for ticker in MAANG
    cleaned_data = dropmissing(dataframes_dict[ticker], [:Return, :GSPC_Return])
    model = lm(@formula(Return ~ GSPC_Return), cleaned_data)
    beta = coef(model)[2]
    alpha = coef(model)[1]
    dataframes_dict[ticker][!,"Beta"] .= beta
    dataframes_dict[ticker][!,"Alpha"] .= alpha
    dataframes_dict[ticker][!,"Sharpe_Ratio"] .= sqrt(252) * mean(dataframes_dict[ticker][!,"Return"]) / std(dataframes_dict[ticker][!,"Return"])
end

In [12]:
rows_list = []
for ticker in MAANG
    dict1 = Dict()
    dict1["Ticker_name"] = ticker
    dict1["Beta"] = mean(dataframes_dict[ticker][!,"Beta"])
    dict1["Alpha"] = mean(dataframes_dict[ticker][!,"Alpha"])
    dict1["Sharpe_Ratio"] = mean(dataframes_dict[ticker][!,"Sharpe_Ratio"])
    push!(rows_list, dict1)
end

In [13]:
df = DataFrame()

for row in rows_list
    temp_df = DataFrame(row)
    df = vcat(df, temp_df)
end

df

Row,Alpha,Beta,Sharpe_Ratio,Ticker_name
,Float64,Float64,Float64,String
1,0.00275073,1.83557,2.76483,META
2,0.000299918,1.12279,1.58055,AAPL
3,0.000803329,1.51663,1.65906,AMZN
4,0.00116793,1.252,1.49482,NFLX
5,0.000825884,1.39736,1.72693,GOOGL


In [14]:
println("Best risk weighted return: ", df[findmax(df[!,"Sharpe_Ratio"])[2],:]["Ticker_name"])

Best risk weighted return: META


In [15]:
println("Weighted average beta: ", mean(df[!,"Beta"]))

Weighted average beta: 1.4248693421480896


In [16]:
dataframes_dict["^GSPC"] = filter(row -> isfinite(row[:Return]),dataframes_dict["^GSPC"]);

In [17]:
df = hcat(dataframes_dict["META"][!,"Return"], dataframes_dict["AAPL"][!,"Return"], dataframes_dict["AMZN"][!,"Return"], dataframes_dict["NFLX"][!,"Return"], dataframes_dict["GOOGL"][!,"Return"], dataframes_dict["^GSPC"][!,"Return"])

250×6 Matrix{Float64}:
  0.0129879     0.00902093    0.0256588    …   0.0195997     0.0146425
  0.0278579     0.00790075    0.019587         0.0160866     0.0104523
  0.232824      0.0370626     0.0737994        0.072787      0.0146994
 -0.0118663     0.0243999    -0.084315        -0.0274735    -0.0103546
 -0.00251971   -0.0179288    -0.0117033       -0.0179423    -0.00614046
  0.0298828     0.0192447    -0.000685063  …   0.0460641     0.0128725
 -0.0427409    -0.0176527    -0.0201743       -0.0768301    -0.0110807
 -0.0300387    -0.00691155   -0.018091        -0.0438764    -0.0088298
 -0.0211893     0.000927947  -0.00641284      -0.00463112    0.00219526
  0.0303187     0.0188068     0.0197726        0.000422977   0.0114486
  0.000278677  -0.00422495    0.00160735   …   0.000739876  -0.000280415
 -0.0129262     0.0139034     0.014644         0.0238699     0.00277318
 -0.0266426    -0.0104294    -0.0297549       -0.0147514    -0.0137887
  ⋮                                        ⋱     

In [18]:
cor(df)

6×6 Matrix{Float64}:
 1.0       0.534463  0.594361  0.31462   0.609519  0.587586
 0.534463  1.0       0.423564  0.391779  0.51434   0.706743
 0.594361  0.423564  1.0       0.359034  0.595888  0.600497
 0.31462   0.391779  0.359034  1.0       0.319017  0.418581
 0.609519  0.51434   0.595888  0.319017  1.0       0.59842
 0.587586  0.706743  0.600497  0.418581  0.59842   1.0